In [31]:
# -*- coding: UTF-8 -*-
import numpy as np 
import pandas as pd
import jieba
import jieba.analyse
import codecs
from collections import Counter

# Read files

# Seperate words for the text

In [2]:
from collections import Counter
import jieba
jieba.load_userdict('userdict.txt')

# 创建停用词list
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r').readlines()]
    return stopwords

# 对句子进行分词
def seg_sentence(sentence):
    sentence_seged = jieba.cut(sentence.strip())
    stopwords = stopwordslist('stoped.txt')  # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr



Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/89/qp8kb90s49z7xcdskq2c0qlm0000gn/T/jieba.cache
Loading model cost 0.688 seconds.
Prefix dict has been built succesfully.


# Calculate characters

In [3]:
#Calculate the frequency of roles in certain length text
def calculate_roles(inputs,roles_df):
    roles_dict=dict()
    for line in inputs:
        #print(line)
        line_seg = seg_sentence(line)
        words_list=list(line_seg.split(' '))
        #print(words_list)
        for role in roles_df['character']:
            #print(role)
            if role in words_list:
                if role not in roles_dict:
                    roles_dict[role]=1
                else:
                    roles_dict[role]+=1
                #print(role)
    return roles_dict
    

# seperate chapters and save all chapters into one list

In [4]:
def sep_chapters(text,chapter_df):
    one_chapter=list() #used to save all lines in single chapter
    all_chapters=list() # used to save all chapters into one list
    for line in text:
        for chpater_no in chapter_df['chapters']:
            if chpater_no in line:
                all_chapters.append(one_chapter)
                one_chapter=list()
        one_chapter.append(line)
        #print(one_chapter)
    return all_chapters
    

## Generate role frequency in each chapter

In [ ]:
# Read texts
# Read character file
#text = open('哈利波特与密室(人文版).txt', 'r',encoding="gbk") #加载要处理的文件的路径
roles_df=pd.read_excel('character.xlsx',names=['character'],index=[0])
roles_df.head()
# Read chapter file
chapter_df=pd.read_excel('chapters.xlsx',index=[0])
#chapter_df.head()
# Save the book names in a list
file_names=['哈利波特与魔法石','哈利波特与密室','哈利波特与阿兹卡班的囚徒','哈利波特与火焰杯',
           '哈利波特与凤凰社','哈利波特与混血王子','哈利波特与死亡圣器']

for file_name in file_names:
    chapter_no=1
    file_path = './dataset/哈利波特/'+file_name+'(人文版).txt'
    text = open(file_path, 'r',encoding="utf-8") #加载要处理的文件的路径
    #Seperate chapters for the text
    chapters_list=sep_chapters(text,chapter_df)
    #calculate role numbers for each chapter and save it into a .xlsx file
    for chapter in chapters_list: 
        output_name = './dataset/哈利波特/output/角色统计/'+file_name+' 第'+str(chapter_no)+'章 角色统计.xlsx'
        #print(output_name)
        chapter_freq=calculate_roles(chapter,roles_df)
        result_df=pd.DataFrame(columns = ['角色','出现次数'])
        result_df['角色'] = chapter_freq.keys()
        result_df['出现次数'] = chapter_freq.values()
        result_df.sort_values(by='出现次数',axis=0,ascending=False, inplace=True)
        result_df.to_excel(output_name)
        chapter_no+=1

## Generate role frequencies in each book

In [ ]:
# Read texts
# Read character file
#text = open('哈利波特与密室(人文版).txt', 'r',encoding="gbk") #加载要处理的文件的路径
roles_df=pd.read_excel('character.xlsx',names=['character'],index=[0])
roles_df.head()
# Read chapter file
chapter_df=pd.read_excel('chapters.xlsx',index=[0])
#chapter_df.head()
# Save the book names in a list
file_names=['哈利波特与魔法石','哈利波特与密室','哈利波特与阿兹卡班的囚徒','哈利波特与火焰杯',
           '哈利波特与凤凰社','哈利波特与混血王子','哈利波特与死亡圣器']

for file_name in file_names:
    chapter_no=1
    file_path = './dataset/哈利波特/'+file_name+'(人文版).txt'
    text = open(file_path, 'r',encoding="utf-8") #加载要处理的文件的路径
    output_name = './dataset/哈利波特/output/角色统计/'+file_name+' 角色统计.xlsx'
    #print(output_name)
    result_df=pd.DataFrame(columns = ['角色','出现次数'])
    #Seperate chapters for the text
    chapters_list=sep_chapters(text,chapter_df)
    #calculate role numbers for each chapter and save it into a .xlsx file
    chapter_freq=calculate_roles(text,roles_df)
    #print(chapter_freq)
    result_df['角色'] = chapter_freq.keys()
    result_df['出现次数'] = chapter_freq.values()
    result_df.sort_values(by='出现次数',axis=0,ascending=False, inplace=True)
    result_df.to_excel(output_name)
    chapter_no+=1
    
    

# Generate data of relationships between characters/roles in the book

## Genereate text of each page

In [5]:
# Generate single line data
def line_generating(text):
    word_counting = 1 # line one
    words_in_line = 30 #There are 30 words in a line in the book
    one_line = '' # save the word in a line into a string
    result_lines = '' # save all lines into one list
    for blocks in text:
        #print(blocks)
        for word in blocks:
            if word == '\n':
                #lines.append(one_line)
                word_counting=0
                one_line = ''
            elif word_counting <= 30:
                #print(word)
                one_line = one_line+word
            else:
                result_lines = result_lines + '\n' +one_line
                #print(one_line)
                word_counting=0
                one_line = ''
            #print(one_line)
            word_counting +=1
    return result_lines
        

## Generate lines in each page and save them all in a dict

In [6]:
def page_generating(text_lines):
    #因为每35行是一页，所以我们把每一页的第一行保存起来，
    #就像我们分章节时，我们把‘第一章’，‘第二章‘这样的mark保存起来，这样程序遍历文本时，遇到每页第一行就保存下来
    line_counting = 1 #line one
    lines_in_page = 36 #There are 35 lines in a page of the book
    page_mark = []
    for line in text_lines.split('\n'):   
        if line_counting >= lines_in_page:
            page_mark.append(line)
            line_counting =1
        line_counting += 1
    page_mark_df=pd.DataFrame(columns = ['chapters'])
    page_mark_df['chapters']=page_mark
    return page_mark_df

## Generate the page mark for each page

In [7]:
chapter_no=1
file_path = './dataset/哈利波特/哈利波特与死亡圣器(人文版).txt'
text = open(file_path, 'r',encoding="utf-8") #加载要处理的文件的路径
result_lines = line_generating(text)
result_mark = page_generating(result_lines)
#text.close()
#page_mark_df=pd.DataFrame(columns = ['chapters'])
#page_mark_df['chapters']=page_mark


In [8]:
result_mark.head()

,chapters
0,“星期六……傍晚。”伏地魔重复了一句。他的红眼睛死死
1,“这样更好，”伏地魔说，“他只好在露天转移。要抓住他
2,多年来，他们不是一直口口声声地宣称希望我复出，希望我东山再起
3,。那女人旋转着面对炉火时，用沙哑而恐惧的声音说：“西弗勒斯！
4,和波特臭大粪的淡淡字样；接着他又掏出一个破旧开裂的窥镜和一个


## Seperate text in pages

In [9]:
file_path = './dataset/哈利波特/哈利波特与死亡圣器(人文版).txt'
text = open(file_path, 'r',encoding="utf-8") #加载要处理的文件的路径
pages_list=sep_chapters(text,result_mark)


## Get roles in each pages

In [72]:
roles_df=pd.read_excel('character.xlsx',names=['character'],index=[0])
roles_df.head()
page_roles = []
#calculate role numbers in each page and save them all in a list for now
for chapter in pages_list: 
    #output_name = './dataset/哈利波特/output/角色统计/'+file_name+' 第'+str(chapter_no)+'章 角色统计.xlsx'
    #print(output_name)
    chapter_freq=calculate_roles(chapter,roles_df)
    page_roles.append(list(chapter_freq.keys()))
    

## Get two-role combinations

In [32]:
def get_role_combinations(page_roles):
    role_collections = [] # A container for all two-role combinations
    for roles in page_roles:    
        j= len(roles)
        while j>1:
            j-=1
            i=j
            role_com = set()        
            while i>0:
                i-=1
                #print(roles[i])
                role_com.add(roles[j])
                role_com.add(roles[i])
                role_collections.append(tuple(role_com))
                #print(role_com)
                role_com = set()
    result = Counter(role_collections)
    return result

## Count the combinations

In [74]:
combination_counter=get_role_combinations(page_roles)

## generate the results into formatted files

In [77]:
file = open('哈利波特与死亡圣器人物关系.txt', 'w')
file.write('[')
for relations in combination_counter.keys():
    #print(k)
    file.write('[')
    for person in relations:
        #print(i)
        file.write("'"+person+"',")
        #file.write(',')
    #print(c[k])
    file.write(str(combination_counter[relations])+'],\n')
    
file.write(']')
file.close()